In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("C:/Users/User/我的雲端硬碟 (owen.lin@mutual-boost.com)/MBQ_Tej_v2/MBQ_tej_v2")
import MBQ_tej_v2_manager
import Tool
import numpy as np
import tqdm
import tejapi
from datetime import datetime, timedelta

In [109]:
twse = tejapi.get('TWN/APIPRCD', coid='IX0001',opts={'columns': ['mdate','roi','close_d','vol','low_d','high_d']}, mdate={'gte': '2010-01-01', 'lte':'2025-04-21'}, paginate=True)

In [110]:
twse['roi'] = (twse['roi'] / 100).shift(-2)

# volume timing

In [ ]:
def calc_rank_factor(df: pd.DataFrame, N: int) -> pd.DataFrame:

    df = df.copy()
    rank_list = [np.nan] * N
    for i in range(N, len(df)):
        window = df["vol"].iloc[i - N : i + 1].tolist()
        sorted_window = sorted(window)
        rank_list.append(sorted_window.index(df["vol"].iloc[i]) + 1)  

    df["rank"] = rank_list
    # 標準化： (2·rank − N − 2) / N
    df["rank_factor"] = (2 * df["rank"] - N - 2) / N
    return df


def divide_market(df: pd.DataFrame, C: float) -> pd.DataFrame:
    df = df.copy()
    df["10days_ret"] = df["close_d"].pct_change(10)
    df["market"] = ""

    df.loc[df["10days_ret"] > C, "market"] = "r"
    df.loc[(df["10days_ret"] <= C) & (df["10days_ret"] >= -C), "market"] = "c"
    df.loc[df["10days_ret"] < -C, "market"] = "f"
    return df


def run_market_strategy(
    df: pd.DataFrame,
    N: int,
    C: float,
    Sf: float,
    Sc: float,
    Sr: float,
) -> pd.DataFrame:
    df = df.copy()
    df = calc_rank_factor(df, N)
    df = divide_market(df, C)

    # 不同行情對應的閾值
    S_dict = {"f": Sf, "c": Sc, "r": Sr}

    df["flag"] = 0      # 交易訊號：1 開多、‑1 平倉、0 無動作
    df["position"] = 0  # 持倉狀態：1 多單、0 空手
    position = 0

    df = df.replace("", np.nan).dropna().reset_index(drop=True)

    for i in df.index:
        mkt = df.at[i, "market"]
        rk  = df.at[i, "rank_factor"]

        if rk > S_dict[mkt] and position == 0:
            df.at[i, "flag"] = 1
            position = 1          # 建立多單
        elif rk < S_dict[mkt] and position == 1:
            df.at[i, "flag"] = -1
            position = 0          # 平倉
        # 否則維持現有部位

        df.at[i, "position"] = position

    return df.dropna()


In [149]:
# twse['mdate'] = pd.to_datetime(twse['mdate'])
# twse = twse.set_index('mdate')
df = twse

In [ ]:
def calculate_statistics(df):

    df = df.copy()
    df['net_asset_pct_chg'] = df['net_asset_value'].pct_change(1).fillna(0)

    total_return = df['net_asset_value'].iloc[-1] / df['net_asset_value'].iloc[0] - 1
    annual_return = (1 + total_return) ** (252 / len(df)) - 1
    total_return *= 100
    annual_return *= 100

    df['ex_pct_chg'] = df['net_asset_pct_chg']
    sharp_ratio = df['ex_pct_chg'].mean() * math.sqrt(252) / df['ex_pct_chg'].std()

    df['high_level'] = df['net_asset_value'].cummax()
    df['draw_down'] = df['net_asset_value'] - df['high_level']
    df['draw_down_percent'] = df['draw_down'] / df['high_level'] * 100
    max_draw_down = df['draw_down'].min()
    max_draw_percent = df['draw_down_percent'].min()

    hold_days = df['position'].sum()
    trade_count = df[df['flag'] != 0].shape[0] // 2
    avg_hold_days = int(hold_days / trade_count) if trade_count > 0 else 0

    profit_days = df[df['net_asset_pct_chg'] > 0].shape[0]
    loss_days = df[df['net_asset_pct_chg'] < 0].shape[0]
    winrate_by_day = profit_days / (profit_days + loss_days) * 100 if (profit_days + loss_days) > 0 else 0

    avg_profit_rate_day = df[df['net_asset_pct_chg'] > 0]['net_asset_pct_chg'].mean() * 100
    avg_loss_rate_day = df[df['net_asset_pct_chg'] < 0]['net_asset_pct_chg'].mean() * 100
    avg_profit_loss_ratio_day = avg_profit_rate_day / abs(avg_loss_rate_day) if avg_loss_rate_day != 0 else None

    buy_trades = df[df['flag'] == 1].reset_index(drop=True)
    sell_trades = df[df['flag'] == -1].reset_index(drop=True)

    if not buy_trades.empty and not sell_trades.empty and len(buy_trades) == len(sell_trades):
        result_by_trade = pd.DataFrame({
            'buy': buy_trades['close_d'],
            'sell': sell_trades['close_d'],
        })
        result_by_trade['pct_chg'] = (result_by_trade['sell'] - result_by_trade['buy']) / result_by_trade['buy']

        profit_trades = result_by_trade[result_by_trade['pct_chg'] > 0].shape[0]
        loss_trades = result_by_trade[result_by_trade['pct_chg'] < 0].shape[0]

        max_profit_trade = result_by_trade['pct_chg'].max() * 100
        max_loss_trade = result_by_trade['pct_chg'].min() * 100

        winrate_by_trade = profit_trades / (profit_trades + loss_trades) * 100 if (profit_trades + loss_trades) > 0 else 0
        avg_profit_rate_trade = result_by_trade[result_by_trade['pct_chg'] > 0]['pct_chg'].mean() * 100
        avg_loss_rate_trade = result_by_trade[result_by_trade['pct_chg'] < 0]['pct_chg'].mean() * 100
        avg_profit_loss_ratio_trade = avg_profit_rate_trade / abs(avg_loss_rate_trade) if avg_loss_rate_trade != 0 else None
    else:
        profit_trades = loss_trades = max_profit_trade = max_loss_trade = winrate_by_trade = None
        avg_profit_rate_trade = avg_loss_rate_trade = avg_profit_loss_ratio_trade = None

    statistics_result = {
        'net_asset_value': df['net_asset_value'].iloc[-1],
        'total_return': total_return,
        'annual_return': annual_return,
        'sharp_ratio': sharp_ratio,
        'max_draw_percent': max_draw_percent,
        'hold_days': hold_days,
        'trade_count': trade_count,
        'avg_hold_days': avg_hold_days,
        'profit_days': profit_days,
        'loss_days': loss_days,
        'winrate_by_day': winrate_by_day,
        'avg_profit_rate_day': avg_profit_rate_day,
        'avg_loss_rate_day': avg_loss_rate_day,
        'avg_profit_loss_ratio_day': avg_profit_loss_ratio_day,
        'profit_trades': profit_trades,
        'loss_trades': loss_trades,
        'max_profit_trade': max_profit_trade,
        'max_loss_trade': max_loss_trade,
        'winrate_by_trade': winrate_by_trade,
        'avg_profit_rate_trade': avg_profit_rate_trade,
        'avg_loss_rate_trade': avg_loss_rate_trade,
        'avg_profit_loss_ratio_trade': avg_profit_loss_ratio_trade
    }

    return statistics_result


In [ ]:
df = twse
import cufflinks as cf
cf.go_offline()
df = calc_rank_factor(df, N=20)
df = run_market_strategy(df, N=20, C=0.05, Sf=0.7, Sc=0.4, Sr=0.2)

df['ret'] = df['close_d'].pct_change().fillna(0)
df['strategy_ret'] = df['ret'] * df['position'].shift(1).fillna(0)
df['net_asset_value'] = (1 + df['strategy_ret']).cumprod()

stats = calculate_statistics(df)
# stats["stock"] = stock_id
result_df = pd.DataFrame([stats])  

df['cum_ret'] = df['strategy_ret'].cumsum()

cumsum_df = df[["cum_ret",'roi']]
cumsum_df['roi'] = cumsum_df['roi'].cumsum()

cumsum_df.iplot(
    title=f"Cumulative Return",
    xTitle="Date",
    yTitle="Cumulative Return",
    kind="line",
    asFigure=False
)
result_df

,net_asset_value,total_return,annual_return,sharp_ratio,max_draw_percent,hold_days,trade_count,avg_hold_days,profit_days,loss_days,...,avg_loss_rate_day,avg_profit_loss_ratio_day,profit_trades,loss_trades,max_profit_trade,max_loss_trade,winrate_by_trade,avg_profit_rate_trade,avg_loss_rate_trade,avg_profit_loss_ratio_trade
0,1.910067,91.006688,4.472747,0.530928,-24.756746,1203,496,2,653,550,...,-0.678193,1.001902,271,225,10.184027,-14.167699,54.637097,1.042256,-0.937086,1.112231


# plus rsrs

In [ ]:
import statsmodels.api as sm
def calc_nbeta(df, n=18):
    nbeta = []
    r2 = []
    trade_days = len(df)

    for i in range(trade_days):
        if i < (n - 1):
            nbeta.append(np.nan)
            r2.append(np.nan)
        else:
            try:
                x = df['low_d'].iloc[i-n+1:i+1]
                x = sm.add_constant(x)
                y = df['high_d'].iloc[i-n+1:i+1]
                model = sm.OLS(y, x)
                res = model.fit()
                beta = round(res.params[1], 2)
                nbeta.append(beta)
                r2.append(res.rsquared)
            except:
                nbeta.append(np.nan)
                r2.append(np.nan)

    df1 = df.copy().reset_index(drop=True)
    df1['beta'] = nbeta
    df1['r2'] = r2
    return df1

def calc_stdbeta(df, n=18, m=650):
    df1 = calc_nbeta(df, n)
    df1['stdbeta'] = (
        (df1['beta'] - df1['beta'].rolling(window=m, min_periods=1).mean()) /
        df1['beta'].rolling(window=m, min_periods=1).std()
    )
    return df1
def run_rsrs_strategy(df, N, C, Sf, Sc, Sr, S=0.7):

    df = df.copy()
    df = calc_stdbeta(df)
    df = calc_rank_factor(df, N)
    df = divide_market(df, C)

    S_dict = {'f': Sf, 'c': Sc, 'r': Sr}
    df['flag'] = 0
    df['position'] = 0
    position = 0

    df = df.replace('', np.nan).dropna().reset_index(drop=True)

    for i in df.index:
        mkt = df.loc[i, 'market']
        stdb = df.loc[i, 'stdbeta']
        rank = df.loc[i, 'rank_factor']

        if mkt == 'r':  # 牛市：只看 RSRS
            if stdb > S and position == 0:
                df.loc[i, 'flag'] = 1
                position = 1
            elif stdb < -S and position == 1:
                df.loc[i, 'flag'] = -1
                position = 0

        elif mkt == 'c':  # 震盪市：RSRS + rank_factor 雙條件
            if stdb > S and rank > S_dict[mkt] and position == 0:
                df.loc[i, 'flag'] = 1
                position = 1
            elif stdb < -S and rank < -S_dict[mkt] and position == 1:
                df.loc[i, 'flag'] = -1
                position = 0

        elif mkt == 'f':  # 熊市：RSRS + rank_factor 更嚴格
            if stdb > S and rank > S_dict[mkt] and position == 0:
                df.loc[i, 'flag'] = 1
                position = 1
            elif (stdb < -S or rank < -S_dict[mkt]) and position == 1:
                df.loc[i, 'flag'] = -1
                position = 0

        df.loc[i, 'position'] = position

    return df.dropna()


In [167]:
df = twse
df['low_d'] = df['low_d'].replace(0, np.nan)
df['high_d'] = df['high_d'].replace(0, np.nan)
df = run_rsrs_strategy(df, N=20, C=0.05, Sf=0.7, Sc=0.4, Sr=0.2)

df['ret'] = df['close_d'].pct_change().fillna(0)
df['strategy_ret'] = df['ret'] * df['position'].shift(1).fillna(0)
df['net_asset_value'] = (1 + df['strategy_ret']).cumprod()

stats = calculate_statistics(df)
# stats["stock"] = stock_id
result_df = pd.DataFrame([stats])  

df['cum_ret'] = df['strategy_ret'].cumsum()
df['roi'] = df['roi'].cumsum()  # 大盤累積報酬

import cufflinks as cf
cf.go_offline()
df[['cum_ret', 'roi']].iplot(title="RSRS + Rank 策略累積報酬", xTitle="Date", yTitle="Cumulative Return")
result_df

,net_asset_value,total_return,annual_return,sharp_ratio,max_draw_percent,hold_days,trade_count,avg_hold_days,profit_days,loss_days,...,avg_loss_rate_day,avg_profit_loss_ratio_day,profit_trades,loss_trades,max_profit_trade,max_loss_trade,winrate_by_trade,avg_profit_rate_trade,avg_loss_rate_trade,avg_profit_loss_ratio_trade
0,1.348751,34.875122,2.04349,0.251682,-31.870549,1841,43,42,975,866,...,-0.676693,0.944912,24,19,22.759916,-16.639744,55.813953,5.316417,-4.542943,1.170258


# rsrs research

# std

In [ ]:
def cal_stdbeta(df, n=18):
    df = df.copy()

    if 'low_d' not in df.columns or 'high_d' not in df.columns:
        raise ValueError("需要 low_d 和 high_d 欄位")

    beta = []
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        if i < n - 1:
            beta.append(np.nan)
        else:
            x = df['low_d'].iloc[i-n+1:i+1]
            y = df['high_d'].iloc[i-n+1:i+1]

            if x.isna().any() or y.isna().any():
                beta.append(np.nan)
                continue

            try:
                x = sm.add_constant(x)
                model = sm.OLS(y, x).fit()
                beta.append(model.params[1])
            except:
                beta.append(np.nan)

    df['beta'] = beta

    # 過濾掉 NaN 再計算標準化
    beta_array = df['beta'].dropna().values
    if len(beta_array) == 0:
        print("所有 beta 計算皆為 NaN，請檢查 low_d/high_d")
        df['stdbeta'] = np.nan
        return df

    mu = np.mean(beta_array)
    sigma = np.std(beta_array)

    print(f"beta 平均: {mu:.4f}, 標準差: {sigma:.6f}, 有效樣本: {len(beta_array)}")

    if sigma == 0:
        print("sigma = 0，標準化無法計算")
        df['stdbeta'] = np.nan
        return df

    df['stdbeta'] = (df['beta'] - mu) / sigma

    # 執行簡單策略（建倉 > 0.7、平倉 < -0.7）
    df['flag'] = 0
    df['position'] = 0
    position = 0

    for i in range(len(df) - 1):
        if df['stdbeta'].iloc[i] > 0.7 and position == 0:
            df.loc[i, 'flag'] = 1
            df.loc[i+1, 'position'] = 1
            position = 1
        elif df['stdbeta'].iloc[i] < -0.7 and position == 1:
            df.loc[i, 'flag'] = -1
            df.loc[i+1, 'position'] = 0
            position = 0
        else:
            df.loc[i+1, 'position'] = df.loc[i, 'position']

    return df


In [ ]:
df = twse
df = cal_stdbeta(df, n=18)

df['ret'] = df['close_d'].pct_change().fillna(0)
df['strategy_ret'] = df['ret'] * df['position'].shift(1).fillna(0)
df['net_asset_value'] = (1 + df['strategy_ret']).cumprod()

stats = calculate_statistics(df)
result_df = pd.DataFrame([stats]) 
df['cum_ret'] = df['strategy_ret'].cumsum()
df['roi'] = df['roi'].cumsum()  # 大盤累積報酬

cumsum_df = df[["cum_ret", "roi"]]
cumsum_df.iplot(
    title="RSRS 累積報酬 vs 大盤",
    xTitle="Date",
    yTitle="Cumulative Return",
    kind="line",
    asFigure=False
)
result_df


beta 平均: 0.8953, 標準差: 0.105018, 有效樣本: 3732


,net_asset_value,total_return,annual_return,sharp_ratio,max_draw_percent,hold_days,trade_count,avg_hold_days,profit_days,loss_days,...,avg_loss_rate_day,avg_profit_loss_ratio_day,profit_trades,loss_trades,max_profit_trade,max_loss_trade,winrate_by_trade,avg_profit_rate_trade,avg_loss_rate_trade,avg_profit_loss_ratio_trade
0,1.469317,46.931695,2.620271,0.287512,-29.885838,1881,64,29,1011,870,...,-0.738798,0.924568,38,26,14.634508,-14.139536,59.375,3.860154,-3.8754,0.996066


# r square

In [ ]:
import numpy as np
import statsmodels.api as sm

def cal_better_stdbeta(df, n=18):
    df = df.copy().reset_index(drop=True)

    nbeta = []
    R2 = []

    for i in range(len(df)):
        if i < n - 1:
            nbeta.append(np.nan)
            R2.append(np.nan)
        else:
            x = df['low_d'].iloc[i - n + 1:i + 1]
            y = df['high_d'].iloc[i - n + 1:i + 1]

            if x.isna().any() or y.isna().any():
                nbeta.append(np.nan)
                R2.append(np.nan)
                continue

            x = sm.add_constant(x)
            model = sm.OLS(y, x).fit()
            beta = round(model.params[1], 2)
            r2 = model.rsquared

            nbeta.append(beta)
            R2.append(r2)

    # 標準化 beta
    prebeta = np.array(nbeta)
    sigma = np.nanstd(prebeta)
    mu = np.nanmean(prebeta)
    stdbeta = (prebeta - mu) / sigma

    # 用 R² 作為加權係數
    r2 = np.array(R2)
    better_stdbeta = r2 * stdbeta

    df['beta'] = nbeta
    df['better_stdbeta'] = better_stdbeta
    df['flag'] = 0
    df['position'] = 0
    position = 0

    for i in range(len(df) - 1):  # -1 是為了不越界
        if df['better_stdbeta'].iloc[i] > 0.7 and position == 0:
            df.loc[i, 'flag'] = 1
            df.loc[i + 1, 'position'] = 1
            position = 1
        elif df['better_stdbeta'].iloc[i] < -0.7 and position == 1:
            df.loc[i, 'flag'] = -1
            df.loc[i + 1, 'position'] = 0
            position = 0
        else:
            df.loc[i + 1, 'position'] = df.loc[i, 'position']

    df['net_asset_value'] = (1 + df['close_d'].pct_change().fillna(0) * df['position']).cumprod()

    return df


In [ ]:
df = twse
df = cal_better_stdbeta(df, n=18)

df['ret'] = df['close_d'].pct_change().fillna(0)
df['strategy_ret'] = df['ret'] * df['position'].shift(1).fillna(0)
df['net_asset_value'] = (1 + df['strategy_ret']).cumprod()

stats = calculate_statistics(df)
result_df = pd.DataFrame([stats])
df['cum_ret'] = df['strategy_ret'].cumsum()
df['roi'] = df['roi'].cumsum()  # 大盤累積報酬

cumsum_df = df[["cum_ret", "roi"]]
cumsum_df.iplot(
    title="RSRS 累積報酬 vs 大盤",
    xTitle="Date",
    yTitle="Cumulative Return",
    kind="line",
    asFigure=False
)
result_df


,net_asset_value,total_return,annual_return,sharp_ratio,max_draw_percent,hold_days,trade_count,avg_hold_days,profit_days,loss_days,...,avg_loss_rate_day,avg_profit_loss_ratio_day,profit_trades,loss_trades,max_profit_trade,max_loss_trade,winrate_by_trade,avg_profit_rate_trade,avg_loss_rate_trade,avg_profit_loss_ratio_trade
0,1.504045,50.404531,2.781538,0.302447,-33.706934,1883,54,34,1010,873,...,-0.73213,0.93208,32,22,11.714189,-14.738519,59.259259,4.305209,-4.575067,0.941016


In [ ]:
def cal_right_stdbeta(df, n=16):
    df1 = cal_better_stdbeta(df, n).copy()
    df1 = df1.reset_index(drop=True)
    df1['position'] = 0
    df1['flag'] = 0

    df1['right_stdbeta'] = df1['better_stdbeta'] * df1['beta']

    # 交易策略：右偏 beta > 0.7 建倉，< -0.7 平倉
    position = 0
    for i in range(len(df1) - 1):
        score = df1.loc[i, 'right_stdbeta']

        if score > 0.7 and position == 0:
            df1.loc[i, 'flag'] = 1
            df1.loc[i + 1, 'position'] = 1
            position = 1
        elif score < -0.7 and position == 1:
            df1.loc[i, 'flag'] = -1
            df1.loc[i + 1, 'position'] = 0
            position = 0
        else:
            df1.loc[i + 1, 'position'] = df1.loc[i, 'position']

    df1['net_asset_value'] = (1 + df1['close_d'].pct_change().fillna(0) * df1['position']).cumprod()

    return df1


# 右偏

In [ ]:
df = twse
df = cal_right_stdbeta(df, n=18)

df['ret'] = df['close_d'].pct_change().fillna(0)
df['strategy_ret'] = df['ret'] * df['position'].shift(1).fillna(0)
df['net_asset_value'] = (1 + df['strategy_ret']).cumprod()

stats = calculate_statistics(df)
result_df = pd.DataFrame([stats]) 

df['cum_ret'] = df['strategy_ret'].cumsum()
df['roi'] = df['roi'].cumsum()  # 大盤累積報酬

cumsum_df = df[["cum_ret", "roi"]]
cumsum_df.iplot(
    title="RSRS 累積報酬 vs 大盤",
    xTitle="Date",
    yTitle="Cumulative Return",
    kind="line",
    asFigure=False
)
result_df


,net_asset_value,total_return,annual_return,sharp_ratio,max_draw_percent,hold_days,trade_count,avg_hold_days,profit_days,loss_days,...,avg_loss_rate_day,avg_profit_loss_ratio_day,profit_trades,loss_trades,max_profit_trade,max_loss_trade,winrate_by_trade,avg_profit_rate_trade,avg_loss_rate_trade,avg_profit_loss_ratio_trade
0,1.360192,36.019206,2.089325,0.232658,-27.91743,2050,49,41,1096,954,...,-0.745111,0.921364,30,19,19.310285,-14.738519,61.22449,4.50456,-5.268042,0.855073


# market timing

In [ ]:
def cal_ma_beta(df, n=18):
    df1 = cal_stdbeta(df, n).copy()
    df1 = df1.reset_index(drop=True)

    df1['position'] = 0
    df1['flag'] = 0
    position = 0

    # 確保已有 ma20
    if 'ma20' not in df1.columns:
        df1['ma20'] = df1['close_d'].rolling(20).mean()

    for i in range(5, len(df1) - 1): 
        beta = df1.loc[i, 'stdbeta']
        close_now = df1.loc[i - 1, 'close_d']
        close_before = df1.loc[i - 3, 'close_d']
        ma_now = df1.loc[i - 1, 'ma20']
        ma_before = df1.loc[i - 3, 'ma20']

        if beta > 0.7 and close_now > ma_now and close_before < ma_before and position == 0:
            df1.loc[i, 'flag'] = 1
            df1.loc[i + 1, 'position'] = 1
            position = 1

        elif beta < -0.7 and close_now < ma_now and close_before > ma_before and position == 1:
            df1.loc[i, 'flag'] = -1
            df1.loc[i + 1, 'position'] = 0
            position = 0

        else:
            df1.loc[i + 1, 'position'] = df1.loc[i, 'position']

    df1['net_asset_value'] = (1 + df1['close_d'].pct_change().fillna(0) * df1['position']).cumprod()
    return df1


In [ ]:
df = twse
df = cal_ma_beta(df, n=18)

df['ret'] = df['close_d'].pct_change().fillna(0)
df['strategy_ret'] = df['ret'] * df['position'].shift(1).fillna(0)
df['net_asset_value'] = (1 + df['strategy_ret']).cumprod()

stats = calculate_statistics(df)
result_df = pd.DataFrame([stats])  
df['cum_ret'] = df['strategy_ret'].cumsum()
df['roi'] = df['roi'].cumsum()  # 大盤累積報酬

cumsum_df = df[["cum_ret", "roi"]]
cumsum_df.iplot(
    title="RSRS 累積報酬 vs 大盤",
    xTitle="Date",
    yTitle="Cumulative Return",
    kind="line",
    asFigure=False
)
result_df


beta 平均: 0.8953, 標準差: 0.105018, 有效樣本: 3732


,net_asset_value,total_return,annual_return,sharp_ratio,max_draw_percent,hold_days,trade_count,avg_hold_days,profit_days,loss_days,...,avg_loss_rate_day,avg_profit_loss_ratio_day,profit_trades,loss_trades,max_profit_trade,max_loss_trade,winrate_by_trade,avg_profit_rate_trade,avg_loss_rate_trade,avg_profit_loss_ratio_trade
0,0.937592,-6.240763,-0.432216,-0.032885,-30.630991,858,17,50,439,419,...,-0.642717,0.943038,9,8,21.240965,-18.501492,52.941176,4.968118,-4.675884,1.062498


# rsrs + vol

In [ ]:
def cal_vol_beta(df, n=18):
    df1 = cal_stdbeta(df, n).copy()
    df1 = df1.reset_index(drop=True)

    df1['position'] = 0
    df1['flag'] = 0
    position = 0

    for i in range(10, len(df1) - 1):
        beta_now = df1.loc[i, 'stdbeta']
        beta_series = df1['stdbeta'].iloc[i - 10:i]
        volume_series = df1['vol'].iloc[i - 10:i]

        if beta_series.isna().any() or volume_series.isna().any():
            continue

        corr = beta_series.corr(volume_series)

        # 建倉條件：RSRS 強 + 量價同步放大
        if beta_now > 0.7 and corr > 0 and position == 0:
            df1.loc[i, 'flag'] = 1
            df1.loc[i + 1, 'position'] = 1
            position = 1

        # 平倉條件：RSRS 明顯轉弱
        elif beta_now < -0.7 and position == 1:
            df1.loc[i, 'flag'] = -1
            df1.loc[i + 1, 'position'] = 0
            position = 0

        # 持續持倉
        else:
            df1.loc[i + 1, 'position'] = df1.loc[i, 'position']

    df1['net_asset_value'] = (1 + df1['close_d'].pct_change().fillna(0) * df1['position']).cumprod()
    return df1


In [ ]:
df = twse
df = cal_vol_beta(df, n=18)

df['ret'] = df['close_d'].pct_change().fillna(0)
df['strategy_ret'] = df['ret'] * df['position'].shift(1).fillna(0)
df['net_asset_value'] = (1 + df['strategy_ret']).cumprod()

stats = calculate_statistics(df)
result_df = pd.DataFrame([stats]) 
df['cum_ret'] = df['strategy_ret'].cumsum()
df['roi'] = df['roi'].cumsum()  # 大盤累積報酬

cumsum_df = df[["cum_ret", "roi"]]
cumsum_df.iplot(
    title="RSRS 累積報酬 vs 大盤",
    xTitle="Date",
    yTitle="Cumulative Return",
    kind="line",
    asFigure=False
)
result_df


beta 平均: 0.8953, 標準差: 0.105018, 有效樣本: 3732


,net_asset_value,total_return,annual_return,sharp_ratio,max_draw_percent,hold_days,trade_count,avg_hold_days,profit_days,loss_days,...,avg_loss_rate_day,avg_profit_loss_ratio_day,profit_trades,loss_trades,max_profit_trade,max_loss_trade,winrate_by_trade,avg_profit_rate_trade,avg_loss_rate_trade,avg_profit_loss_ratio_trade
0,1.575903,57.590265,3.104475,0.366829,-24.690119,1497,57,26,814,683,...,-0.706416,0.930151,34,23,9.992449,-14.139536,59.649123,3.628641,-3.305886,1.09763
